In [5]:
import json
from concurrent.futures import ThreadPoolExecutor

import boto3
from botocore import UNSIGNED
from botocore.config import Config
import pandas as pd
from web3 import Web3

from mainnet_launch.database.schema.full import RebalancePlans, Destinations, DexSwapSteps, Tokens

from mainnet_launch.database.schema.postgres_operations import (
    get_full_table_as_orm,
    get_full_table_as_df,
    insert_avoid_conflicts,
    get_subset_not_already_in_column,
)

from mainnet_launch.constants import ALL_AUTOPOOLS, AutopoolConstants, USDC, WETH, AUTO_ETH


autopool = AUTO_ETH

s3_client = boto3.client("s3", config=Config(signature_version=UNSIGNED))
rebalance_plan_df: list[RebalancePlans] = get_full_table_as_df(
    RebalancePlans, where_clause=RebalancePlans.chain_id == autopool.chain.chain_id
    )
rebalance_plan_df

2025-05-27 14:02:42,270 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-05-27 14:02:42,270 INFO sqlalchemy.engine.Engine SELECT pg_catalog.pg_class.relname 
FROM pg_catalog.pg_class JOIN pg_catalog.pg_namespace ON pg_catalog.pg_namespace.oid = pg_catalog.pg_class.relnamespace 
WHERE pg_catalog.pg_class.relname = %(table_name)s AND pg_catalog.pg_class.relkind = ANY (ARRAY[%(param_1)s, %(param_2)s, %(param_3)s, %(param_4)s, %(param_5)s]) AND pg_catalog.pg_table_is_visible(pg_catalog.pg_class.oid) AND pg_catalog.pg_namespace.nspname != %(nspname_1)s
2025-05-27 14:02:42,271 INFO sqlalchemy.engine.Engine [cached since 244.7s ago] {'table_name': <sqlalchemy.sql.elements.TextClause object at 0x12821b880>, 'param_1': 'r', 'param_2': 'p', 'param_3': 'f', 'param_4': 'v', 'param_5': 'm', 'nspname_1': 'pg_catalog'}
2025-05-27 14:02:42,272 INFO sqlalchemy.engine.Engine 
            SELECT *
            FROM rebalance_plans
            WHERE rebalance_plans.chain_id = 1
        
2025-05-27 14:02

,file_name,datetime_generated,autopool_vault_address,chain_id,solver_address,rebalance_type,destination_out,token_out,destination_in,token_in,...,in_dest_apr,in_dest_adj_apr,apr_delta,swap_offset_period,num_candidate_destinations,candidate_destinations_rank,projected_swap_cost,projected_net_gain,projected_gross_gain,projected_slippage
0,rebalance_plan_1727897421_0x0A2b94F6871c1D7A32...,2024-10-02 19:30:21+00:00,0x0A2b94F6871c1D7A32Fe58E1ab5e6deA2f114E56,1,0x952D7a7eB2e0804d37d9244BE8e47341356d2f5D,FromIdle,0x896eCc16Ab4AFfF6cE0765A5B924BaECd7Fa455a,0xe080027Bd47353b5D1639772b4a75E9Ed3658A0d,0x0A2b94F6871c1D7A32Fe58E1ab5e6deA2f114E56,0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2,...,0.000000,0.092285,0.092285,50.0,12.0,0.0,1.989009e-01,1.003935,1.202836,0.199381
1,rebalance_plan_1736580621_0x0A2b94F6871c1D7A32...,2025-01-11 07:30:21+00:00,0x0A2b94F6871c1D7A32Fe58E1ab5e6deA2f114E56,1,0x952D7a7eB2e0804d37d9244BE8e47341356d2f5D,FromIdle,0xc4Eb861e7b66f593482a3D7E8adc314f6eEDA30B,0x88794C65550DeB6b4087B7552eCf295113794410,0x0A2b94F6871c1D7A32Fe58E1ab5e6deA2f114E56,0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2,...,0.000000,0.086773,0.086773,46.0,5.0,0.0,5.690695e-02,1.388123,1.445030,0.041353
2,rebalance_plan_1739302221_0x0A2b94F6871c1D7A32...,2025-02-11 19:30:21+00:00,0x0A2b94F6871c1D7A32Fe58E1ab5e6deA2f114E56,1,0x952D7a7eB2e0804d37d9244BE8e47341356d2f5D,FromIdle,0xbA1462f43c6f60ebD1C62735c94E428aD073E01A,0xC8Eb2Cf2f792F77AF0Cd9e203305a585E588179D,0x0A2b94F6871c1D7A32Fe58E1ab5e6deA2f114E56,0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2,...,0.000000,0.089842,0.089842,42.0,4.0,0.0,2.562382e-02,0.408126,0.433750,0.059284
3,rebalance_plan_1737682221_0x0A2b94F6871c1D7A32...,2025-01-24 01:30:21+00:00,0x0A2b94F6871c1D7A32Fe58E1ab5e6deA2f114E56,1,None,None,None,None,None,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,rebalance_plan_1743481821_0x0A2b94F6871c1D7A32...,2025-04-01 04:30:21+00:00,0x0A2b94F6871c1D7A32Fe58E1ab5e6deA2f114E56,1,None,None,None,None,None,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3383,rebalance_plan_1748049286_0xa7569A44f348d3D70d...,2025-05-24 01:14:46+00:00,0xa7569A44f348d3D70d8ad5889e50F78E33d80D35,1,0xD02b50CFc6c2903bF13638B28D081ad11515B6f9,Pool2Pool,0x7876F91BB22148345b3De16af9448081E9853830,0x9Fb7b4477576Fe5B32be4C1843aFB1e55F251B33,0x7583b1589aDD33320366A48A92794D77763FAE9e,0x390f3595bCa2Df7d23783dFd126427CCeb997BF4,...,0.057203,0.079587,0.022384,14.0,3.0,0.0,3.237657e-11,86.053494,86.053494,0.032362
3384,rebalance_plan_1748117686_0xa7569A44f348d3D70d...,2025-05-24 20:14:46+00:00,0xa7569A44f348d3D70d8ad5889e50F78E33d80D35,1,0xD02b50CFc6c2903bF13638B28D081ad11515B6f9,Pool2Pool,0x7876F91BB22148345b3De16af9448081E9853830,0x9Fb7b4477576Fe5B32be4C1843aFB1e55F251B33,0x7583b1589aDD33320366A48A92794D77763FAE9e,0x390f3595bCa2Df7d23783dFd126427CCeb997BF4,...,0.059091,0.077252,0.018161,14.0,3.0,0.0,3.580927e-11,69.749488,69.749488,0.035797
3385,rebalance_plan_1748340886_0xa7569A44f348d3D70d...,2025-05-27 10:14:46+00:00,0xa7569A44f348d3D70d8ad5889e50F78E33d80D35,1,0xD02b50CFc6c2903bF13638B28D081ad11515B6f9,Pool2Pool,0x7876F91BB22148345b3De16af9448081E9853830,0x9Fb7b4477576Fe5B32be4C1843aFB1e55F251B33,0xbB2d2dd491204a86ec10a1a6972F940B34fE060e,0x4f493B7dE8aAC7d55F71853688b1F7C8F0243C85,...,0.059546,0.066662,0.007116,14.0,3.0,0.0,3.003755e-11,17.705725,17.705725,0.030034
3386,rebalance_plan_1748031286_0xa7569A44f348d3D70d...,2025-05-23 20:14:46+00:00,0xa7569A44f348d3D70d8ad5889e50F78E33d80D35,1,0xD02b50CFc6c2903bF13638B28D081ad11515B6f9,FromIdle,0x7876F91BB22148345b3De16af9448081E9853830,0x9Fb7b4477576Fe5B32be4C1843aFB1e55F251B33,0xa7569A44f348d3D70d8ad5889e50F78E33d80D35,0xA0b86991c6218b36c1d19D4a2e9Eb0cE3606eB48,...,0.000000,0.078102,0.078102,14.0,3.0,0.0,7.613385e-12,137.985141,137.985141,0.015000


In [4]:
from mainnet_launch.data_fetching.tokemak_subgraph import fetch_autopool_rebalance_events_from_subgraph
rebalance_event_df = fetch_autopool_rebalance_events_from_subgraph(autopool)
rebalance_event_df

,transactionHash,timestamp,blockNumber,tokenInAmount,tokenIn,tokenOutAmount,tokenOut,destinationInAddress,destinationOutAddress,tokenInAddress,tokenOutAddress
20,0xf4b0efb2e5d69ef7b7d000b4860a9d86b76013d9a042...,1726602143,20772483,99.961014,{'id': '0xc8eb2cf2f792f77af0cd9e203305a585e588...,100.000000,{'id': '0xc02aaa39b223fe8d0a0e5c4f27ead9083c75...,0xE382BBd32C4E202185762eA433278f4ED9E6151E,0x0A2b94F6871c1D7A32Fe58E1ab5e6deA2f114E56,0xC8Eb2Cf2f792F77AF0Cd9e203305a585E588179D,0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2
6,0xfdb95550d0720ff1f6ca2ff7331d1eb3194fe85ee8ab...,1726632143,20774967,314.380624,{'id': '0xc8eb2cf2f792f77af0cd9e203305a585e588...,314.500371,{'id': '0xc02aaa39b223fe8d0a0e5c4f27ead9083c75...,0xE382BBd32C4E202185762eA433278f4ED9E6151E,0x0A2b94F6871c1D7A32Fe58E1ab5e6deA2f114E56,0xC8Eb2Cf2f792F77AF0Cd9e203305a585E588179D,0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2
162,0xaa013ce865573532459a55487fb809c3e93edeed05af...,1726676555,20778643,144.676019,{'id': '0x6951bdc4734b9f7f3e1b74afebc670c736a0...,146.034135,{'id': '0xc02aaa39b223fe8d0a0e5c4f27ead9083c75...,0xd96E943098B2AE81155e98D7DC8BeaB34C539f01,0x0A2b94F6871c1D7A32Fe58E1ab5e6deA2f114E56,0x6951bDC4734b9f7F3E1B74afeBC670c736A0EDB6,0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2
404,0x1e5ddbb3a2e56692c1c261b05b482298c1cd6bd6bcf3...,1726714667,20781802,133.437132,{'id': '0x6951bdc4734b9f7f3e1b74afebc670c736a0...,134.705433,{'id': '0xc02aaa39b223fe8d0a0e5c4f27ead9083c75...,0xd96E943098B2AE81155e98D7DC8BeaB34C539f01,0x0A2b94F6871c1D7A32Fe58E1ab5e6deA2f114E56,0x6951bDC4734b9f7F3E1B74afeBC670c736A0EDB6,0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2
13,0xfb4645f276bf56d5ff8fef87301516ccb8ed0841a5db...,1726752383,20784928,335.615197,{'id': '0xc8eb2cf2f792f77af0cd9e203305a585e588...,335.831303,{'id': '0xc02aaa39b223fe8d0a0e5c4f27ead9083c75...,0xE382BBd32C4E202185762eA433278f4ED9E6151E,0x0A2b94F6871c1D7A32Fe58E1ab5e6deA2f114E56,0xC8Eb2Cf2f792F77AF0Cd9e203305a585E588179D,0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2
...,...,...,...,...,...,...,...,...,...,...,...
265,0x70c1b88035c7001cb055b2218381494dcf0fdea4cf73...,1748223095,22563717,51.730527,{'id': '0x6951bdc4734b9f7f3e1b74afebc670c736a0...,52.567729,{'id': '0x88794c65550deb6b4087b7552ecf29511379...,0xe4433D00Cf48BFE0C672d9949F2cd2c008bffC04,0xc4Eb861e7b66f593482a3D7E8adc314f6eEDA30B,0x6951bDC4734b9f7F3E1B74afeBC670c736A0EDB6,0x88794C65550DeB6b4087B7552eCf295113794410
260,0x7356f89ae7b56795daec5ca6476ee2ce97f8124b784d...,1748255495,22566385,38.219307,{'id': '0xc8eb2cf2f792f77af0cd9e203305a585e588...,38.175659,{'id': '0x88794c65550deb6b4087b7552ecf29511379...,0xbA1462f43c6f60ebD1C62735c94E428aD073E01A,0xc4Eb861e7b66f593482a3D7E8adc314f6eEDA30B,0xC8Eb2Cf2f792F77AF0Cd9e203305a585E588179D,0x88794C65550DeB6b4087B7552eCf295113794410
5,0xfdf67dc6d3e6c48b79c91f295646fd48837d06fbbf5c...,1748287895,22569065,28.657303,{'id': '0xc8eb2cf2f792f77af0cd9e203305a585e588...,28.631744,{'id': '0x88794c65550deb6b4087b7552ecf29511379...,0xbA1462f43c6f60ebD1C62735c94E428aD073E01A,0xc4Eb861e7b66f593482a3D7E8adc314f6eEDA30B,0xC8Eb2Cf2f792F77AF0Cd9e203305a585E588179D,0x88794C65550DeB6b4087B7552eCf295113794410
127,0xbc8853882a7d05247c9fb42f8a942fdab269b60f7fe6...,1748320307,22571756,21.137299,{'id': '0x6951bdc4734b9f7f3e1b74afebc670c736a0...,21.473808,{'id': '0x88794c65550deb6b4087b7552ecf29511379...,0xe4433D00Cf48BFE0C672d9949F2cd2c008bffC04,0xc4Eb861e7b66f593482a3D7E8adc314f6eEDA30B,0x6951bDC4734b9f7F3E1B74afeBC670c736A0EDB6,0x88794C65550DeB6b4087B7552eCf295113794410


In [13]:
rebalance_event_df['datetime_executed'] = pd.to_datetime(rebalance_event_df['timestamp'].astype(int), unit='s')
rebalance_event_df = rebalance_event_df.sort_values('datetime_executed')
rebalance_plan_df = rebalance_plan_df.sort_values('datetime_generated')

In [15]:
rebalance_event_df

,transactionHash,timestamp,blockNumber,tokenInAmount,tokenIn,tokenOutAmount,tokenOut,destinationInAddress,destinationOutAddress,tokenInAddress,tokenOutAddress,datetime_executed
20,0xf4b0efb2e5d69ef7b7d000b4860a9d86b76013d9a042...,1726602143,20772483,99.961014,{'id': '0xc8eb2cf2f792f77af0cd9e203305a585e588...,100.000000,{'id': '0xc02aaa39b223fe8d0a0e5c4f27ead9083c75...,0xE382BBd32C4E202185762eA433278f4ED9E6151E,0x0A2b94F6871c1D7A32Fe58E1ab5e6deA2f114E56,0xC8Eb2Cf2f792F77AF0Cd9e203305a585E588179D,0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2,2024-09-17 19:42:23
6,0xfdb95550d0720ff1f6ca2ff7331d1eb3194fe85ee8ab...,1726632143,20774967,314.380624,{'id': '0xc8eb2cf2f792f77af0cd9e203305a585e588...,314.500371,{'id': '0xc02aaa39b223fe8d0a0e5c4f27ead9083c75...,0xE382BBd32C4E202185762eA433278f4ED9E6151E,0x0A2b94F6871c1D7A32Fe58E1ab5e6deA2f114E56,0xC8Eb2Cf2f792F77AF0Cd9e203305a585E588179D,0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2,2024-09-18 04:02:23
162,0xaa013ce865573532459a55487fb809c3e93edeed05af...,1726676555,20778643,144.676019,{'id': '0x6951bdc4734b9f7f3e1b74afebc670c736a0...,146.034135,{'id': '0xc02aaa39b223fe8d0a0e5c4f27ead9083c75...,0xd96E943098B2AE81155e98D7DC8BeaB34C539f01,0x0A2b94F6871c1D7A32Fe58E1ab5e6deA2f114E56,0x6951bDC4734b9f7F3E1B74afeBC670c736A0EDB6,0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2,2024-09-18 16:22:35
404,0x1e5ddbb3a2e56692c1c261b05b482298c1cd6bd6bcf3...,1726714667,20781802,133.437132,{'id': '0x6951bdc4734b9f7f3e1b74afebc670c736a0...,134.705433,{'id': '0xc02aaa39b223fe8d0a0e5c4f27ead9083c75...,0xd96E943098B2AE81155e98D7DC8BeaB34C539f01,0x0A2b94F6871c1D7A32Fe58E1ab5e6deA2f114E56,0x6951bDC4734b9f7F3E1B74afeBC670c736A0EDB6,0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2,2024-09-19 02:57:47
13,0xfb4645f276bf56d5ff8fef87301516ccb8ed0841a5db...,1726752383,20784928,335.615197,{'id': '0xc8eb2cf2f792f77af0cd9e203305a585e588...,335.831303,{'id': '0xc02aaa39b223fe8d0a0e5c4f27ead9083c75...,0xE382BBd32C4E202185762eA433278f4ED9E6151E,0x0A2b94F6871c1D7A32Fe58E1ab5e6deA2f114E56,0xC8Eb2Cf2f792F77AF0Cd9e203305a585E588179D,0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2,2024-09-19 13:26:23
...,...,...,...,...,...,...,...,...,...,...,...,...
265,0x70c1b88035c7001cb055b2218381494dcf0fdea4cf73...,1748223095,22563717,51.730527,{'id': '0x6951bdc4734b9f7f3e1b74afebc670c736a0...,52.567729,{'id': '0x88794c65550deb6b4087b7552ecf29511379...,0xe4433D00Cf48BFE0C672d9949F2cd2c008bffC04,0xc4Eb861e7b66f593482a3D7E8adc314f6eEDA30B,0x6951bDC4734b9f7F3E1B74afeBC670c736A0EDB6,0x88794C65550DeB6b4087B7552eCf295113794410,2025-05-26 01:31:35
260,0x7356f89ae7b56795daec5ca6476ee2ce97f8124b784d...,1748255495,22566385,38.219307,{'id': '0xc8eb2cf2f792f77af0cd9e203305a585e588...,38.175659,{'id': '0x88794c65550deb6b4087b7552ecf29511379...,0xbA1462f43c6f60ebD1C62735c94E428aD073E01A,0xc4Eb861e7b66f593482a3D7E8adc314f6eEDA30B,0xC8Eb2Cf2f792F77AF0Cd9e203305a585E588179D,0x88794C65550DeB6b4087B7552eCf295113794410,2025-05-26 10:31:35
5,0xfdf67dc6d3e6c48b79c91f295646fd48837d06fbbf5c...,1748287895,22569065,28.657303,{'id': '0xc8eb2cf2f792f77af0cd9e203305a585e588...,28.631744,{'id': '0x88794c65550deb6b4087b7552ecf29511379...,0xbA1462f43c6f60ebD1C62735c94E428aD073E01A,0xc4Eb861e7b66f593482a3D7E8adc314f6eEDA30B,0xC8Eb2Cf2f792F77AF0Cd9e203305a585E588179D,0x88794C65550DeB6b4087B7552eCf295113794410,2025-05-26 19:31:35
127,0xbc8853882a7d05247c9fb42f8a942fdab269b60f7fe6...,1748320307,22571756,21.137299,{'id': '0x6951bdc4734b9f7f3e1b74afebc670c736a0...,21.473808,{'id': '0x88794c65550deb6b4087b7552ecf29511379...,0xe4433D00Cf48BFE0C672d9949F2cd2c008bffC04,0xc4Eb861e7b66f593482a3D7E8adc314f6eEDA30B,0x6951bDC4734b9f7F3E1B74afeBC670c736A0EDB6,0x88794C65550DeB6b4087B7552eCf295113794410,2025-05-27 04:31:47


In [14]:
rebalance_plan_df

,file_name,datetime_generated,autopool_vault_address,chain_id,solver_address,rebalance_type,destination_out,token_out,destination_in,token_in,...,in_dest_apr,in_dest_adj_apr,apr_delta,swap_offset_period,num_candidate_destinations,candidate_destinations_rank,projected_swap_cost,projected_net_gain,projected_gross_gain,projected_slippage
140,rebalance_plan_1726517778_0x0A2b94F6871c1D7A32...,2024-09-16 20:16:18+00:00,0x0A2b94F6871c1D7A32Fe58E1ab5e6deA2f114E56,1,0x952D7a7eB2e0804d37d9244BE8e47341356d2f5D,FromIdle,0xE382BBd32C4E202185762eA433278f4ED9E6151E,0xC8Eb2Cf2f792F77AF0Cd9e203305a585E588179D,0x0A2b94F6871c1D7A32Fe58E1ab5e6deA2f114E56,0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2,...,0.00000,0.100466,0.100466,50.0,13.0,0.0,0.141324,2.325664,2.466988,0.141324
2278,rebalance_plan_1726517907_0xE800e3760FC20aA98c...,2024-09-16 20:18:27+00:00,0xE800e3760FC20aA98c5df6A9816147f190455AF3,1,0x067cB94576aBBDB3527055f3c8CaAAc9668364AD,FromIdle,0x2F7e096a400ded5D02762120b39A3aA4ABA072a4,0x7761b6E0Daa04E70637D81f1Da7d186C205C2aDE,0xE800e3760FC20aA98c5df6A9816147f190455AF3,0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2,...,0.00000,0.104658,0.104658,50.0,6.0,0.0,0.098612,0.978934,1.077546,0.126344
1391,rebalance_plan_1726518264_0x6dC3ce9C57b2013134...,2024-09-16 20:24:24+00:00,0x6dC3ce9C57b20131347FDc9089D740DAf6eB34c5,1,0x9cf7E7d9F02CaEdcE2286199C0c20dd92580433d,FromIdle,0x2F7e096a400ded5D02762120b39A3aA4ABA072a4,0x7761b6E0Daa04E70637D81f1Da7d186C205C2aDE,0x6dC3ce9C57b20131347FDc9089D740DAf6eB34c5,0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2,...,0.00000,0.098349,0.098349,50.0,12.0,0.0,0.162259,1.386278,1.548538,0.135603
1136,rebalance_plan_1726519182_0x6dC3ce9C57b2013134...,2024-09-16 20:39:42+00:00,0x6dC3ce9C57b20131347FDc9089D740DAf6eB34c5,1,0x9cf7E7d9F02CaEdcE2286199C0c20dd92580433d,FromIdle,0x2F7e096a400ded5D02762120b39A3aA4ABA072a4,0x7761b6E0Daa04E70637D81f1Da7d186C205C2aDE,0x6dC3ce9C57b20131347FDc9089D740DAf6eB34c5,0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2,...,0.00000,0.098064,0.098064,50.0,12.0,0.0,0.165486,1.404926,1.570412,0.135963
478,rebalance_plan_1726539377_0x0A2b94F6871c1D7A32...,2024-09-17 02:16:17+00:00,0x0A2b94F6871c1D7A32Fe58E1ab5e6deA2f114E56,1,0x952D7a7eB2e0804d37d9244BE8e47341356d2f5D,FromIdle,0xE382BBd32C4E202185762eA433278f4ED9E6151E,0xC8Eb2Cf2f792F77AF0Cd9e203305a585E588179D,0x0A2b94F6871c1D7A32Fe58E1ab5e6deA2f114E56,0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2,...,0.00000,0.100524,0.100524,50.0,13.0,0.0,0.141062,3.143942,3.285004,0.141062
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2421,rebalance_plan_1748347241_0x35911af1B570E26f66...,2025-05-27 12:00:41+00:00,0x35911af1B570E26f668905595dEd133D01CD3E5a,1,None,None,None,None,None,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3323,rebalance_plan_1748352624_0x0A2b94F6871c1D7A32...,2025-05-27 13:30:24+00:00,0x0A2b94F6871c1D7A32Fe58E1ab5e6deA2f114E56,1,0x952D7a7eB2e0804d37d9244BE8e47341356d2f5D,Pool2Pool,0xbA1462f43c6f60ebD1C62735c94E428aD073E01A,0xC8Eb2Cf2f792F77AF0Cd9e203305a585E588179D,0xc4Eb861e7b66f593482a3D7E8adc314f6eEDA30B,0x88794C65550DeB6b4087B7552eCf295113794410,...,0.02871,0.045080,0.016370,38.0,3.0,0.0,0.002720,0.017160,0.019880,0.016854
3341,rebalance_plan_1748356249_0x6dC3ce9C57b2013134...,2025-05-27 14:30:49+00:00,0x6dC3ce9C57b20131347FDc9089D740DAf6eB34c5,1,None,None,None,None,None,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2422,rebalance_plan_1748358041_0x35911af1B570E26f66...,2025-05-27 15:00:41+00:00,0x35911af1B570E26f668905595dEd133D01CD3E5a,1,None,None,None,None,None,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [16]:
rebalance_event_df['tokenOutAmount']

20     100.000000
6      314.500371
162    146.034135
404    134.705433
13     335.831303
          ...    
265     52.567729
260     38.175659
5       28.631744
127     21.473808
309     45.064088
Name: tokenOutAmount, Length: 456, dtype: float64

In [17]:
rebalance_plan_df['amount_out']

140     100.000000
2278     78.050515
1391    119.657376
1136    121.714203
478     100.000000
           ...    
2421           NaN
3323     16.105356
3341           NaN
2422           NaN
3387     99.795595
Name: amount_out, Length: 3388, dtype: float64